In [189]:
import glob
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', None)  

In [247]:
# Get stats and combine into position dataframes

# Function to find positional files, read them, add the week of the game, and return as one single dataframe
def position_df(file):
    l = []
    i = 1
    files = glob.glob(file)
    files = sorted(files)
    print(files)
    for file in files:
        df = pd.read_csv(file)
        df['week'] = i
        l.append(df)
        i += 1
    l = pd.concat(l)
    return(l)

In [312]:
# Get dataframe for each position
passing = position_df('stats_19/fantasy-stats-passing*.csv')
rec_rush = position_df('stats_19/fantasy-stats-rec*.csv')
dst = position_df('stats_19/fantasy-stats-dst*.csv')
idps = position_df('stats_19/fantasy-stats-defense*.csv')
#kickers = position_df('stats_19/fantasy-stats-kicking*.csv')

# Create schedule dataframe, used to add in opponents
schedule = pd.read_csv('schedule.csv')

['stats_19/fantasy-stats-passing_19_wk1.csv', 'stats_19/fantasy-stats-passing_19_wk2.csv', 'stats_19/fantasy-stats-passing_19_wk3.csv', 'stats_19/fantasy-stats-passing_19_wk4.csv', 'stats_19/fantasy-stats-passing_19_wk5.csv', 'stats_19/fantasy-stats-passing_19_wk6.csv']
['stats_19/fantasy-stats-rec-rush_19_wk1.csv', 'stats_19/fantasy-stats-receiving_19_wk2.csv', 'stats_19/fantasy-stats-receiving_19_wk3.csv']
['stats_19/fantasy-stats-dst_19_wk1.csv', 'stats_19/fantasy-stats-dst_19_wk2.csv', 'stats_19/fantasy-stats-dst_19_wk3.csv']


In [250]:
# Create dict of team abvs and full names, fix errors in dict
team_long = np.unique(list([schedule['Winner/tie'],schedule['Loser/tie']]))
team_abv = passing['team'].unique()
team_abv = np.sort(team_abv)
teams_dict = dict(zip(team_long, team_abv))
chargers = teams_dict['Los Angeles Rams']
teams_dict['Los Angeles Rams'] = teams_dict['Los Angeles Chargers']
teams_dict['Los Angeles Chargers'] = chargers
niners = teams_dict['Seattle Seahawks']
teams_dict['Seattle Seahawks'] = teams_dict['San Francisco 49ers']
teams_dict['San Francisco 49ers'] = niners

In [252]:
# create list of lists of opponenets for each row of the passing df by week
q =[]
for i in range(1,passing['week'].max()+1):
    b = []
    week = schedule[schedule['Week'] == i]
    winners = week['Winner/tie']
    losers = week['Loser/tie']
    matchups = dict(zip(winners,losers))
    matchups2 = dict(zip(losers,winners))
    matchups.update(matchups2)
    for k,v in matchups.items():
        c = []
        c.append(teams_dict[k])
        c.append(teams_dict[v])
        b.append(c)
    b = dict(b)
    z = []
    passing_wk = passing[passing['week'] == i]
    for i in passing_wk['team']:
        z.append(b[i])
    q.append(z)

1
2
3
4
5
6


In [279]:
def create(df):
    q =[]
    for i in range(1,df['week'].max()+1):
        b = []
        week = schedule[schedule['Week'] == i]
        winners = week['Winner/tie']
        losers = week['Loser/tie']
        matchups = dict(zip(winners,losers))
        matchups2 = dict(zip(losers,winners))
        matchups.update(matchups2)
        for k,v in matchups.items():
            c = []
            c.append(teams_dict[k])
            c.append(teams_dict[v])
            b.append(c)
        b = dict(b)
        z = []
        df_wk = df[df['week'] == i]
        for i in df_wk['team']:
            z.append(b[i])
        q.append(z)
    # Flattend list and append new column to passing df    
    flattened  = [val for sublist in q for val in sublist]
    df['opp'] = flattened
    return df

In [339]:
rec_rush_df = create(rec_rush)

In [340]:
rec_rush_df

,player,team,games,position,recTarg,recRec,recRec40s,recYds,recYds100Games,recTds,recTd40s,recDrops,catch,depth,ypt,ypr,rac,rzRecTarg,rzRecRec,rzRecTds,rzRecTargPct,rzRecRecPct,rzRecTdPct,ezRecTarg,ezRecTds,ezRecTargPct,ezRecRecPct,ezRecTdPct,rushCarries,rush40s,rushYds,rushYds100Games,rushTds,rushTd40s,ypc,yac,rushTa,tat,fumbles,rzRushCarries,rzRushTds,rzRushPct,rzRushTdPct,i5RushCarries,i5RushTds,i5RushPct,i5RushTdPct,patConversions,patAttempts,fantasyPts,ptsPerSnap,ptsPerTouch,week,opp
3,A.J. Brown,TEN,1,WR,5,3,0,25,0,0,0,2,60.0,6.00,5.00,8.33,2.33,0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0.00,0.00,0,0.00,0,0,0,0.0,0,0,0,0.0,0.0,0,0,5.5,0.20,1.83,2,CLV
95,A.J. Brown,TEN,1,WR,4,3,2,100,1,0,0,0,75.0,14.25,25.00,33.33,18.33,1,0,0,25.0,0.0,0.0,1,0,25.0,0.0,0.0,0,0,0,0,0,0,0.00,0.00,0,0.00,0,0,0,0.0,0,0,0,0.0,0.0,0,0,13.0,0.52,4.33,1,CHI
99,A.J. Brown,TEN,1,WR,5,1,0,4,0,0,0,0,20.0,13.80,0.80,4.00,0.00,0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0.00,0.00,0,0.00,0,0,0,0.0,0,0,0,0.0,0.0,0,0,1.4,0.04,1.40,3,CLV
205,Aaron Jones,GB,1,RB,7,5,0,40,0,0,0,0,71.4,2.29,5.71,8.00,9.00,1,1,0,14.3,20.0,0.0,0,0,0.0,0.0,0.0,22,0,110,1,1,0,5.00,2.23,3,0.14,0,3,1,13.6,100,1,1,4.5,100.0,0,0,26.0,0.62,0.96,2,GB
197,Aaron Jones,GB,1,RB,1,1,0,4,0,0,0,0,100.0,1.00,4.00,4.00,3.00,0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,10,0,19,0,2,0,1.90,1.30,1,0.10,0,3,2,30.0,100,2,1,20.0,50.0,0,0,15.3,0.73,1.39,3,ATL
188,Aaron Jones,GB,1,RB,1,1,0,0,0,0,0,0,100.0,-2.00,0.00,0.00,2.00,0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,13,0,39,0,0,0,3.00,2.38,4,0.31,0,0,0,0.0,0,0,0,0.0,0.0,0,0,4.9,0.13,0.35,1,BLT
70,Adam Humphries,TEN,1,WR,2,2,0,-1,0,0,0,0,100.0,-4.50,-0.50,-0.50,4.00,0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,1,0,1,0,0,0,1.00,2.00,0,0.00,0,0,0,0.0,0,0,0,0.0,0.0,0,0,2.0,0.06,0.67,2,DEN
55,Adam Humphries,TEN,1,WR,1,1,0,5,0,0,0,0,100.0,5.00,5.00,5.00,0.00,0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0.00,0.00,0,0.00,0,0,0,0.0,0,0,0,0.0,0.0,0,0,1.5,0.07,1.50,1,LA
65,Adam Humphries,TEN,1,WR,9,6,0,93,0,0,0,1,66.7,10.56,10.33,15.50,2.83,1,1,0,11.1,16.7,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0.00,0.00,0,0.00,0,0,0,0.0,0,0,0,0.0,0.0,0,0,15.3,0.28,2.55,3,WAS
326,Adam Shaheen,CHI,1,TE,3,3,0,24,0,0,0,0,100.0,2.33,8.00,8.00,5.67,0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0.00,0.00,0,0.00,0,0,0,0.0,0,0,0,0.0,0.0,0,0,5.4,0.27,1.80,2,SEA


In [334]:
rec_rush_df.sort_values('player', inplace=True)

In [355]:
y = rec_rush_df[['player','fantasyPts']]

In [336]:
def prep(df):
    df2 = df.drop('team', axis=1)
    df2 = df2.drop('games', axis=1)
    y = df2['fantasyPts']
    df2 = df2.drop('fantasyPts', axis=1)
    df2 = df2.drop('week', axis=1)
    df2 = df2.sort_values('player')
    opp = df2['opp']
    df2 = df2.drop('opp', axis=1)
    df2 = df2.fillna(0)
    df_avg = df.groupby('player').agg('mean')
    df_avg = df_avg.drop('games', axis=1)
    df_avg = df_avg.drop('fantasyPts', axis=1)
    df_avg = df_avg.drop('week', axis=1)
    df_avg = df_avg.fillna(0)
    # Set index for update, update each row with each avg, and add the opponents
    df2.set_index('player', inplace=True)
    df2.update(df_avg)
    df2['opp'] = opp.values
    #Replace oppoenets with dummies for ML algorithm
    df3 = pd.get_dummies(df2, drop_first=True)
    return(df3)

In [342]:
X = prep(rec_rush_df)

In [367]:
# ML algorithm

In [343]:
kfold = KFold(n_splits=10, random_state=7)
model = Ridge()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 24)
scoring = 'neg_mean_squared_error'
ridge = Ridge(alpha = 0.5, normalize=True).fit(X_train, y_train)
ridge.score(X_test, y_test)

0.5990681122988564

In [346]:
ridge.predict(X_test)

array([ 1.04505229e+01,  8.80609608e-01,  1.13162337e+00,  1.09736358e+01,
        6.81485725e-01,  1.95742136e+00, -1.19679616e+00,  1.88322437e+01,
        2.02992401e+00,  6.06024022e+00,  5.00549346e+00,  2.56633310e+00,
        4.62316689e+00,  2.12267500e+00,  3.18084389e-01,  3.44247078e-01,
        1.34294843e+00,  9.53530195e-01,  1.12904161e+01,  5.95225067e+00,
        2.34506918e+00,  1.72258632e-01,  6.57606387e+00,  1.10194489e+01,
        1.45941153e+01,  1.76779518e+01, -6.65527065e-02,  6.96479410e+00,
        5.89145011e+00,  1.04908773e+01,  8.43490358e+00,  7.54629283e+00,
        2.45015482e+00,  3.84621219e-01,  2.97183581e+00,  1.46390422e+00,
       -1.09173920e+00,  3.95702485e+00,  8.75655984e+00,  1.76779518e+01,
        7.85667271e+00,  1.80435171e+00,  3.18084389e-01,  1.40760644e+00,
        4.54488038e+00,  2.91872728e+00,  5.00773472e+00,  2.12958794e-01,
        2.18641143e+00,  9.09896582e+00,  1.32692363e+01,  2.22674585e-01,
        1.04257218e+01,  

In [347]:
ridge = Ridge(alpha = 0.5, normalize=True).fit(X_train, y_train)

In [348]:
ridge.predict(X_test)

array([ 1.04505229e+01,  8.80609608e-01,  1.13162337e+00,  1.09736358e+01,
        6.81485725e-01,  1.95742136e+00, -1.19679616e+00,  1.88322437e+01,
        2.02992401e+00,  6.06024022e+00,  5.00549346e+00,  2.56633310e+00,
        4.62316689e+00,  2.12267500e+00,  3.18084389e-01,  3.44247078e-01,
        1.34294843e+00,  9.53530195e-01,  1.12904161e+01,  5.95225067e+00,
        2.34506918e+00,  1.72258632e-01,  6.57606387e+00,  1.10194489e+01,
        1.45941153e+01,  1.76779518e+01, -6.65527065e-02,  6.96479410e+00,
        5.89145011e+00,  1.04908773e+01,  8.43490358e+00,  7.54629283e+00,
        2.45015482e+00,  3.84621219e-01,  2.97183581e+00,  1.46390422e+00,
       -1.09173920e+00,  3.95702485e+00,  8.75655984e+00,  1.76779518e+01,
        7.85667271e+00,  1.80435171e+00,  3.18084389e-01,  1.40760644e+00,
        4.54488038e+00,  2.91872728e+00,  5.00773472e+00,  2.12958794e-01,
        2.18641143e+00,  9.09896582e+00,  1.32692363e+01,  2.22674585e-01,
        1.04257218e+01,  

In [349]:
y_test

347    10.5
366    -1.6
163     0.1
286     2.7
182     0.0
65      2.5
265     0.0
48     10.1
336     0.0
116     0.7
62      1.8
303     1.1
252     2.4
267     0.0
194     0.0
118     0.0
170     3.7
98      0.0
15     10.8
230     4.4
245     0.8
270     0.0
104     9.6
217    12.7
335     7.4
29     22.5
260     0.0
32      0.0
368     0.0
286     2.0
       ... 
36      5.0
273     2.8
347    15.9
215     9.7
60     10.0
64      3.3
99      7.1
107     0.0
310     3.6
335     0.0
68      1.9
103     5.8
262     0.0
71      5.8
116     0.0
44     20.0
8       1.6
191    15.9
248     0.4
101     2.3
232     5.7
195    11.3
157     0.0
369     4.5
201     1.9
72      6.9
16      2.0
332     0.0
370    17.0
319     2.4
Name: fantasyPts, Length: 218, dtype: float64

In [350]:
X_scaled = scale(X)
#passing3 = np.array(passing2)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.2, random_state = 42)
ridge = Ridge(alpha = 0.5, normalize=True).fit(X_train, y_train)
ridge.score(X_test, y_test)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by the scale function.
  """Entry point for launching an IPython kernel.


0.5756550181402087

In [351]:
ridge.predict(X_test)

array([ 4.37639801e+00, -1.64511015e-01,  1.01829258e+01,  1.79545047e+00,
        1.69539429e+01,  3.81085988e+00,  9.11544956e+00,  7.12097623e+00,
        9.70439296e-01,  2.72659283e+00,  1.75001219e+01, -5.97524809e-01,
        5.16468604e+00,  2.75230366e+00,  1.29051396e+01,  1.71276067e+01,
        6.74356338e-01, -4.55432518e-01,  1.41764548e+01,  2.26769685e+00,
        1.41108672e+01,  8.02138294e+00,  8.42621193e+00,  4.37808231e+00,
       -2.18118029e-01,  1.64913120e+01, -4.72333820e-01,  5.31613262e+00,
       -8.76280292e-01,  6.74601218e-01,  1.58949675e+01,  2.98535988e+00,
        4.80606709e+00,  6.66632867e+00,  1.06179799e+01,  5.68201194e-02,
        3.82229022e+00,  2.15775908e+01,  1.37251538e+01, -5.53093472e-01,
        2.43913125e+00,  1.36760028e+01,  4.00398289e+00,  6.60756128e+00,
       -1.09246859e+00,  1.32614178e+01,  1.41920376e+01,  7.74379379e+00,
        1.62766277e+01,  1.23582195e+01,  8.82694243e+00,  9.80578406e+00,
        5.42586076e+00,  

In [274]:
y_test

28    18.4
17     3.1
25    48.2
7     28.2
9      0.8
56    -7.8
10     1.6
20    26.5
18     8.9
6     19.3
18     0.0
30     0.0
54     0.0
7     23.3
10     0.0
39     0.0
19    49.3
22     0.0
53     0.0
37     0.0
1    -12.2
11    13.6
31    26.7
63    -0.5
42    30.8
8     39.2
35    23.1
33    12.8
4      0.0
16    30.7
      ... 
38     0.0
32    36.6
9     11.3
28    27.2
27    23.1
2     25.8
7     21.0
37     0.0
23     0.0
19    45.4
1     20.9
3     38.6
30     9.6
41     0.0
23    16.9
37     0.0
12     0.0
31    -1.0
55    38.1
0     29.2
17    13.4
18    35.3
59     3.0
15     0.0
24     0.0
19    68.7
8     11.5
64    38.7
48     0.0
14    -2.4
Name: fantasyPts, Length: 64, dtype: float64

In [ ]:
# Calculate points per game by player

In [ ]:
# Add defense rank to the player row

In [ ]:
# Add metric to value players ability (EPA?)

In [ ]:
# Use metric and defense to train model, test accuracy

In [ ]:
# Add in value for DFS leagues

In [ ]:
# Search and optimize lineups for the best value / most points per lineup